In [1]:
# Imports

import numpy as np
from kanon.tables import HTable
from kanon.units import Sexagesimal
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u

In [2]:
# Calcul du jour
# 3 Juillet 1327

year = 1327
month = 7
day = 3

month_days = [31,59,90,120,151,181,212,243,273,304,334,365]

def qtsp_from_date(y, m, d):
    ndays = int((year-1) * 365.25 + month_days[month-2] + (day - 1))
    q, p = divmod(ndays, 60)
    q, s = divmod(q, 60)
    q, t = divmod(q, 60)
    return (q, t, s, p)

days = qtsp_from_date(year, month, day)
days


(2, 14, 35, 4)

In [3]:
# Définition d'une fonction pour calculer une position à partir d'une table de mouvement moyen

def position_from_table(qtsp, tab, radix, zodiac_offset=4, width=9):
    # Additions à partir de la racine

    result = radix

    # Additions à partir des jours
    with set_precision(pmode=PrecisionMode.MAX):
        for i, v in enumerate(qtsp):
            result += tab.get(v) >> i + zodiac_offset

    # Conversion en quantité degrés avec signe zodiaque

    result %= Sexagesimal(6, 0) * u.degree
    return result

In [4]:
# Position moyenne du soleil

# Lecture de la table depuis DISHAS
tab_mean_motion = HTable.read(193, format="dishas")
tab_mean_motion

Days,Entries
d,deg
object,object
1,"59,08,19,37,19,13,56 ;"
2,"01,58,16,39,14,38,27,52 ;"
3,"02,57,24,58,51,57,41,48 ;"
4,"03,56,33,18,29,16,55,44 ;"
5,"04,55,41,38,06,36,09,40 ;"
6,"05,54,49,57,43,55,23,36 ;"
7,"06,53,58,17,21,14,37,32 ;"
8,"07,53,06,36,58,33,51,28 ;"


In [5]:
# Calcul de la position moyenne à partir des jours, de la table, et de la racine

mean_sun_pos = position_from_table(days, tab_mean_motion, Sexagesimal("4,38;21,0,30,28") * u.degree)
mean_sun_pos

<BasedQuantity 01,47 ; 58,21,32,17,28,35,44 deg>

In [6]:
# Étoiles fixes

tab_fixed_stars = HTable.read(236, format="dishas")

mean_fixed_star_pos = position_from_table(days, tab_fixed_stars, 0)
mean_fixed_star_pos

<BasedQuantity 09 ; 44,44,33,52,35,08,48 deg>

In [7]:
# Accès et recès, auge

tab_access_recess = HTable.read(237, format="dishas")

access_recess_pos = position_from_table(days, tab_access_recess, Sexagesimal("5,59;12,34")*u.degree, zodiac_offset=2, width=7)
access_recess_pos

<BasedQuantity 01,07 ; 25,45,56,14,16 deg>

In [8]:
# Équation accès et recès

tab_eq_access_recess = HTable.read(238, format="dishas")

with set_precision(pmode=PrecisionMode.MAX, tmode=TruncatureMode.ROUND):
    eq_access_recess = tab_eq_access_recess.get(access_recess_pos.value)
eq_access_recess

<BasedQuantity 08 ; 18,18,36,54,20 deg>

In [9]:
# Apogée du soleil
with set_precision(pmode=PrecisionMode.MAX, tmode=TruncatureMode.ROUND):
    solar_apogee_pos = mean_fixed_star_pos + eq_access_recess + Sexagesimal("1,11;25,23") * u.degree
solar_apogee_pos

<BasedQuantity 01,29 ; 28,26,10,46,55,08,48 deg>

In [10]:
# Argument moyen du soleil
with set_precision(pmode=PrecisionMode.MAX, tmode=TruncatureMode.ROUND):
    mean_arg_sun = mean_sun_pos + (Sexagesimal(6,0) * u.degree if mean_sun_pos < solar_apogee_pos else 0) - solar_apogee_pos
mean_arg_sun

<BasedQuantity 18 ; 29,55,21,30,33,26,56 deg>

In [11]:
# Équation du soleil

tab_eq_sun = HTable.read(19, format="dishas")
with set_precision(pmode=PrecisionMode.MAX, tmode=TruncatureMode.ROUND):
    eq_sun = tab_eq_sun.get(mean_arg_sun.value)
eq_sun

<BasedQuantity 00 ; 39,33,20,10,31,40,48 deg>

In [12]:
# Vraie position du soleil

with set_precision(pmode=PrecisionMode.CUSTOM, tmode=TruncatureMode.ROUND, custom_precision=2):
    true_pos_sun = mean_sun_pos + (Sexagesimal(6,0) * u.degree if mean_sun_pos < eq_sun else 0) - eq_sun
true_pos_sun

<BasedQuantity 01,47 ; 18,49 deg>